In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import json

directory_path = '/content/drive/MyDrive/HgpQ/APOBEK1/'  # Ho caricato la cartella dei json su drive

sequences = []
edit_positions = []

for fname in os.listdir(directory_path):
    if fname.endswith('.json'):
        # print(fname)
        gene = json.load(open(directory_path + fname, 'r'))
        start,end = gene['Endpoints'] # posizioni inizio/fine del gene
        editable = gene['Editable_Positions']  # posizioni che possono subire editing nel gene
        pos = [x['Pos']-start for x in editable]  # posizioni della sequenza dna che possono subire editing
        sequence = gene['Sequence']  # sequenza di dna del gene
        if gene['Gene_Direction'] < 0:
            sequence = sequence[::-1]  # inversione ordine della sequenza
        # Sanity check, all marked editable positions should contain a cytosine
        try:
            for p in pos:
                if sequence[p] != 'C':
                    raise ValueError(f'Wrong character at position {p} in gene {fname}')
        except ValueError as ex:
            print(ex)

        # storing task data
        sequences.append(sequence)
        edit_positions.append(pos)


In [ ]:
import pandas as pd


d = {'sequence': sequences, 'edit_positions': edit_positions}
df = pd.DataFrame(data=d)
df.head()

,sequence,edit_positions
0,TGTAGTTTCCGCCGCGTTTATGGCCGCGTTAAGTCTGAGTGCCGCT...,"[1481, 1547, 5044, 5057, 151013, 152476, 17046..."
1,CAAAGCCACCCTGGGTCACCTACATACTAAGGGAATAAACGCCACT...,[425]
2,ACTGTGGCTCCCGAAGCTGCTTCGGGCCCCAGCGGTGAGGAAGAAG...,"[529, 8621, 8648, 8658, 22156, 33299, 44351, 7..."
3,AGAATTTCGTCAAAGTCTGGTCCGACGTAACCCTCTCGGGAAATCC...,"[16072, 26659]"
4,AAAATCTTATAAGTAGTTTCGAAATAAATATTAACAAGGTTTGACC...,"[5045, 5164]"


In [ ]:
df.to_csv("/content/drive/MyDrive/C_edit.csv", index=False)